In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd

In [6]:
user='G'

if user=='S':
    mandat_dep=pd.read_csv("C:/Users/sylva/Desktop/git/Projet-Python-deputes/liste_mandat.csv")#on récupère le csv de la liste des mandats
    df_depu=pd.read_csv("C:/Users/sylva/Desktop/git/Projet-Python-deputes/listeDepute_complete.csv")

if user=='G':
    mandat_dep=pd.read_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/liste_mandatdef.csv") 
    df_depu=pd.read_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/listeDepute_complete.csv")

<h1> I. Base mandat

In [7]:
debut_mandat=[]
for dates in mandat_dep["Mandat"]:
    dates=str(dates).split()
    debut_mandat.append(' '.join(dates[1:4]))
    
mandat_dep["Debut Mandat"]=np.array(debut_mandat)
mandat_dep.head(2)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Nom,Debut Mandat
0,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,Damien Meslot,19 juin 2002
1,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,Damien Meslot,20 juin 2007


In [8]:
df_depu[["Nom","Né le"]].head(2)
mandat_dep[["Nom","Debut Mandat"]].head(2)
age_dep=mandat_dep.set_index('Nom').join(df_depu[["Nom","Né le"]].set_index('Nom'), on='Nom')
mandat_dep.reset_index()
mois={'janvier':1,'février':2, 'mars':3, 'avril':4, 'mai':5, 'juin':6, 'juillet':7, 'août':8, 'septembre':9, 'octobre':10, 'novembre':11, 'décembre':12}
def transtype_date(x):
    x=str(x)        
    date_texte=x.split() 
    if len(date_texte)>1 :
        if  date_texte[1] in (mois.keys()):
            if date_texte[0]=="1er":
                date_texte[0]='1'
            date_texte[1]=str(mois[date_texte[1]])
            date_nombre='/'.join(date_texte)
            return pd.to_datetime(date_nombre)
    else:
        return pd.NaT


age_dep["Date de naissance"]=age_dep["Né le"].apply(lambda x: transtype_date(x))
age_dep["Date debut mandat"]=age_dep["Debut Mandat"].apply(lambda x: transtype_date(x))
age_dep.head(2)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Debut Mandat,Né le,Date de naissance,Date debut mandat
Nom,,,,,,,,,,
Damien Meslot,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,19 juin 2002,11 novembre 1964,1964-11-11,2002-06-19
Damien Meslot,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,20 juin 2007,11 novembre 1964,1964-11-11,2007-06-20


In [9]:
age_dep['age depute']=age_dep['Date debut mandat']-age_dep['Date de naissance']#on crée la variable age au debut du mandat, en jour
age_dep['age en année']=age_dep['age depute'].apply(lambda x: float(x.days / 365.25))#et on la transforme en années
age_dep=age_dep[['Département','Groupe', 'Législature', 'Mandat', 'Régime politique', 'age en année','Date debut mandat']]

In [10]:
age_dep.head(2)

,Département,Groupe,Législature,Mandat,Régime politique,age en année,Date debut mandat
Nom,,,,,,,
Damien Meslot,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,37.601643,2002-06-19
Damien Meslot,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,42.603696,2007-06-20


<h1> II.Table député

<h3> Nombre de mandat par député

In [11]:
nb_mandat=age_dep.groupby("Nom").count()["age en année"]
nb_mandat=pd.DataFrame(nb_mandat).rename(columns={"age en année":"nombre de mandat"})
nb_mandat.head()

,nombre de mandat
Nom,
"(Louis, Camille, Alexandre, Marie Dubochet",1
"Aaron, Jean-François Crassous",1
Abbas Gueye,1
Abbes Moulessehoul,1
Abdallah Tebib,1


In [12]:
df_depu_b=df_depu.join(nb_mandat,on="Nom",lsuffix=" ")
df_depu_b.sample(5)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort,nombre de mandat
9015,9015,"Léon, Louis, Geneviève Journault",24 février 1827,Seine,Yvelines,5.0
13537,13537,Jean-Baptiste Bernardin,16 octobre 1764,Deux-Sèvres,Inconnu,1.0
6876,6876,"Emile, Paul, Auguste Cère",13 mars 1863,Seine,Paris,3.0
7169,7169,"Joseph, Marie, Henri Masclanis",25 mars 1858,Gers,Gers,1.0
10939,10939,"Régis, Auguste, Casimir Demézange",20 juillet 1800,Manche,Manche,1.0


In [13]:
premier_mandat=age_dep.groupby("Nom")["age en année"].min()
premier_mandat.head()

Nom
(Louis, Camille, Alexandre, Marie Dubochet    45.689254
Aaron, Jean-François Crassous                 49.267625
Abbas Gueye                                   37.470226
Abbes Moulessehoul                            38.811773
Abdallah Tebib                                60.766598
Name: age en année, dtype: float64

In [14]:
age_dep=age_dep.reset_index()

In [15]:
premier_mandat_dpt=age_dep.groupby("Nom")[["Département","Nom"]].head(1)
premier_mandat_dpt=pd.DataFrame(premier_mandat_dpt).rename(columns={"Département":"premier dpt"}).set_index("Nom")
premier_mandat_dpt.head(3)

,premier dpt
Nom,
Damien Meslot,Territoire-de-Belfort
Christine Cloarec-Le Nabour,Ille-et-Vilaine
Pascal Terrasse,Ardèche


<h3> Caractéristiques du premier mandat

In [16]:
df_depu_b=df_depu_b.join(premier_mandat,on="Nom",lsuffix="")
df_depu_b=df_depu_b.join(premier_mandat_dpt,on="Nom",lsuffix="")

In [17]:
df_depu_b.sample(5)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort,nombre de mandat,age en année,premier dpt
1461,1461,Philippe Sanmarco,16 février 1947,Inconnu,Inconnu,3.0,34.433949,Bouches-du-Rhône
1700,1700,Aloys Geoffroy,10 avril 1944,Moselle,Inconnu,1.0,48.336756,Meurthe-et-Moselle
933,933,Jean-Claude Leclabart,4 décembre 1954,Somme,Inconnu,0.0,NaN,Somme
5149,5149,Henri Bourlois,9 juin 1893,Inconnu,Inconnu,1.0,31.162218,Seine
3001,3001,Roger Combrisson,22 février 1922,Essonne,Inconnu,4.0,45.026694,Essonne


In [18]:
df_depu_b["élection locale"]=(" "+df_depu_b["Département de naissance"])==df_depu_b["premier dpt"]

In [19]:
df_depu_b.sample(5)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort,nombre de mandat,age en année,premier dpt,élection locale
7365,7365,"Adolphe, Jean, Marie, Cyprien, Maurice Pédebidou",16 décembre 1854,Hautes-Pyrénées,Vienne,2.0,38.677618,Hautes-Pyrénées,True
3795,3795,Victor Rault,6 janvier 1911,Côtes,Côtes,1.0,47.498973,Côtes-du-Nord,False
5052,5052,"René, Louis Cance",29 avril 1895,Cantal,Var,6.0,50.477755,Seine-Inférieure,False
10005,10005,"Eugène, Abel, François Caroillon de Vandeul",14 janvier 1812,Seine,Haute-Marne,1.0,37.327858,Haute-Marne,False
13530,13530,"Pierre, Antoine, Paul Delafenestre",1er novembre 1764,Charente-Inférieure,Charente-Maritime,1.0,51.895962,Charente-Inférieure,True


<h1> III. Base département

In [20]:
dep_possible=list(mandat_dep['Département'].unique())#une liste de tous les départements
dep_possible.remove(" Département d'élection inconnu")
dep_possible.remove(np.nan)

In [21]:
dep_possible[:3]

[' Territoire-de-Belfort', ' Ille-et-Vilaine', ' Ardèche']

In [22]:
def pourcentageTerroir(basedep) :#une fonction qui donne le pourcentage de mandats occupés par des députés nés dans leur département d'élection
    basedep_1=basedep[basedep['Département']==' '+basedep['Département de naissance']]
    nbreterroir=basedep_1[basedep_1['Département de naissance']!='Inconnu']['Département'].count()
    i=basedep[basedep['Département de naissance']!='Inconnu']['Département'].count()
    if i!=0:
        pourcTerroir=nbreterroir/i
    else:
        return np.nan
    return pourcTerroir

In [23]:
df=age_dep.reset_index()

In [24]:
dept_naiss=age_dep.set_index('Nom').join(df_depu[["Nom","Né le", "Département de naissance"]].set_index('Nom'), on='Nom')
dept_naiss=dept_naiss.reset_index()
dept_naiss.head()

,Nom,Département,Groupe,Législature,Mandat,Régime politique,age en année,Date debut mandat,Né le,Département de naissance
0,Damien Meslot,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,37.601643,2002-06-19,11 novembre 1964,Inconnu
1,Damien Meslot,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,42.603696,2007-06-20,11 novembre 1964,Inconnu
2,Damien Meslot,Territoire-de-Belfort,Union pour un mouvement populaire,XIVe législature,Du 20 juin 2012 au 20 juin 2017,Cinquième République - Assemblée nationale,47.605749,2012-06-20,11 novembre 1964,Inconnu
3,Christine Cloarec-Le Nabour,Ille-et-Vilaine,La République en Marche,XVe législature (Législature en cours),Depuis le 21 juin 2017 (mandat en cours),Cinquième République - Assemblée nationale,NaN,NaT,28 octobre 1964,Inconnu
4,Pascal Terrasse,Ardèche,Socialiste,XIe législature,Du 1er juin 1997 au 18 juin 2002,Cinquième République - Assemblée nationale,32.197125,1997-01-06,26 octobre 1964,Gard


In [27]:

w,x,y,z,u=[],[],[],[],[]
for dep in dep_possible :
    w+=[df[df['Département']==dep]['age en année'].mean()]#on prend la moyenne d'age dans le département
    x+=[len(df[df['Département']==dep])]#on compte le nombre total de mandat dans le département
    y+=[len(df[df['Département']==dep]['Nom'].unique())]#Puis le nombre de députés différent par dpt
    z+=[x[-1]/y[-1]] #Ceci est le nombre moyen de mandat par député
    u+=[pourcentageTerroir(dept_naiss[dept_naiss["Département"]==dep])]
    
    
carac_dep_bis=np.array([w, x, y, z,u])#on crée un dataframe avec les informations
table_depmt = pd.DataFrame(carac_dep_bis, index=['age moyen des deputes', 'nombre_mandat', 'nombre député','nombre moyen de mandats par député', 'locaux'],columns=dep_possible)
table_depmt.T.sample(10)

,age moyen des deputes,nombre_mandat,nombre député,nombre moyen de mandats par député,locaux
Citoyens français du Maroc,45.954825,6.0,5.0,1.200000,0.000000
Gard,47.677780,366.0,175.0,2.091429,0.745098
Mont-Blanc,43.130094,48.0,34.0,1.411765,0.000000
Bas-Rhin,47.690881,424.0,197.0,2.152284,0.750000
Départements de l'Empire français aujourd'hui en Belgique,44.775862,98.0,83.0,1.180723,0.000000
Bouches-du-Rhône,48.846335,607.0,263.0,2.307985,0.000000
Charente-Maritime,49.981711,114.0,45.0,2.533333,0.019231
Vosges,48.636473,355.0,159.0,2.232704,0.670487
Territoire français des Affars et des Issas,39.978097,4.0,3.0,1.333333,0.000000
Moselle,48.664374,426.0,184.0,2.315217,0.548571


In [30]:

table_depmt.to_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/table_dpmt.csv")
